## konlpy로 synop 명사 추출
## CountVectorizer, cosine_similarity

1. synop내 명사만으로는 유사도 계산 부정확
2. 제목, 태그, 대분류 완료 후  다시 계산
3. 불용어 제거
4. 다른 방송국 전체 데이터
5. 유사도 계산도 트레이닝이 필요한 것인가?
6. TfidfVectorizer은 다른 nb에 작성(-> TF, IDF 계산 식에 가중치 부여 가능여부?)
7. 각 idx에 추천 epi idx column 추가

In [73]:
import pandas as pd
import numpy as np
from konlpy.tag import *
#from ast import literal_eval

In [74]:
metadata = pd.read_csv('data_nountag_ebs.csv', encoding='cp949')

In [75]:
metadata.head()

,idx,tv,title,epi,dates,links,synop,tag
0,0,EBS,다큐프라임,비밀의 땅 파미르 2부 비밀의 땅 숨겨진 강,2018.09.11,http://www.ebs.co.kr/tv/showcourseId=BP0PAPB00...,아시아의 고산지대인 파미르 텐샨히말라야 의 자연과 사람들의 삶을 알아본다. 중앙아시...,파미르 텐샨 히말라야 사람들 아랄해 아무다리아강 시르다리아강 주요 빙하 타지키스탄 ...
1,1,EBS,다큐프라임,비밀의 땅 파미르 1부 세계의 지붕,2018.09.10,http://www.ebs.co.kr/tv/showcourseId=BP0PAPB00...,세계의 지붕 파미르에 서식하는 다양한 야생동물들의 생태소개 1부에서는 파미르에 서식...,서식 세계 지붕 파미르 야생 야생동물 지역 포식자 늑대 중심 눈표범 마르코폴 동물 ...
2,2,EBS,다큐프라임,위대한 로마 제작노트 로마는 하루아침에 이루어지지 않았다,2018.09.05,http://www.ebs.co.kr/tv/showcourseId=BP0PAPB00...,위대한 로마의 촬영 기간은 총 6주로 3주간 튀니지에서 재연 촬영을 했고 나머지 ...,로마 6주 폼페이 사건 최대 제국 역사 시간 황제 우리
3,3,EBS,다큐프라임,제국의 도시 폼페이,2018.09.04,http://www.ebs.co.kr/tv/showcourseId=BP0PAPB00...,폼페이를 첨단 그래픽과 3D 입체영상으로 완벽 복원한다. 콜로세움이 완성되어 가던...,폼페이
4,4,EBS,다큐프라임,황제들의 정치무대 콜로세움,2018.09.03,http://www.ebs.co.kr/tv/showcourseId=BP0PAPB00...,로마 최대 원형경기장 콜로세움을 첨단 그래픽과 3D 입체영상으로 완벽 복원한다.,NaN


In [76]:
metadata['synop'][755]

'삼동초등학교 교육실험공교육 살리기 프로젝트 180일 간의 기록. 아이들이 달라졌다. 부모의 경제력과 아이의 학력이 비례한다는 얘기는 어제 오늘 이야기가 아니다. 공교육보다 사교육을 믿고 의지하는 것이 오늘의 현실이다. 아이들은 지금 사교육시장으로 내몰리고 있다. EBS다큐프라임에서는 위와 같은 현실을 반영해 교육실험 프로젝트를 선보인다. 약 6개월 간의 촬영을 통해 제작된 본 프로그램은 성공적인 공교육의 사례인 일본 아키타秋田현의 교육방법을 통해 우리나라 공교육에 대해 다시 한 번 생각해 보고자 한다.아키타 현은 평균소득 취업률 모두 일본 최하위인 가난한 현이다. 하지만 2007년 43년 만에 부활한 전국학력평가시험에서 아키타현은 3년 연속 전국 1위라는 작은 기적을 만들어 냈다. 당연한 것을 당연하게 하고 기초 생활습관에 충실하고 학교와 지역사회가 힘을 모아 아이들을 길러내는 아키타현의 교육.이런 아키타의 교육방법을 우리나라 경상남도 남해 삼동초등학교에 도입해 180일 동안 아이들의 변화를 지켜보는 교육실험을 진행했다. 180일 동안 학생들의 변화 과정을 살펴봄으로써 새로운 공교육의 방향성을 제시하고 우리 교육의 희망 메시지를 담는다.  1부 아키타에서 배우다 3년 연속 일본 전국학력평가 전국 1위인 아키타현은 43년 전 전국학력평가시험에서 일본 전체 45개 현 중 43등을 기록했던 지역이다. 그 후 교육입현아키타라는 슬로건 걸고 약 반세기에 걸친 학교 학부모 지역사회가 함께하는 교육을 통해 43년 만에 작은 기적을 만들어 낸다.그 중에서도 공부 잘 하는 학교인 하치모리 소학교. 그 곳에서 배운 것은 노트필기 팀티칭 수업 서로서로 배우기 집에서 복습하기 그리고 교육철학이었다. 위의 방법을 도입해 보기 위해서 우리나라 경상남도 남해에 위치 전교생 76명의 작고 예쁜 남해 삼동초등학교를 찾아간다. 그와 동시에 前 교육부장관인 문용린 서울대 명예 교수와 김청택 서울대 교수 조주연 서울교육대 교수 안진훈 연세대 코칭아카데미 교수로 이루어진 자문단을 구성. 이들과 

In [77]:
len(metadata['synop'])

926

In [155]:
#이 노트북에선 안씀
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
#features = ['epi','synop','tag','big']

# Synop에서 명사 추출 konlpy Hannanum

In [80]:
han = Hannanum()
#kkma = Kkma()
#okt = Okt()

In [81]:
# 메타데이터의 synop을 synoplist에 넣은 후 명사 추출
synoplist = metadata['synop']
synoplist.head()

0    아시아의 고산지대인 파미르 텐샨히말라야 의 자연과 사람들의 삶을 알아본다. 중앙아시...
1    세계의 지붕 파미르에 서식하는 다양한 야생동물들의 생태소개 1부에서는 파미르에 서식...
2     위대한 로마의 촬영 기간은 총 6주로 3주간 튀니지에서 재연 촬영을 했고 나머지 ...
3    폼페이를 첨단 그래픽과 3D 입체영상으로 완벽 복원한다.  콜로세움이 완성되어 가던...
4         로마 최대 원형경기장 콜로세움을 첨단 그래픽과 3D 입체영상으로 완벽 복원한다.
Name: synop, dtype: object

In [82]:
# 불용어 제거 안함.
han.nouns(synoplist[0])

['아시아',
 '고산지대',
 '파미르',
 '텐샨히말라야',
 '의',
 '자연',
 '사람',
 '삶',
 '중앙아시아',
 '거대',
 '호수',
 '아랄해',
 '지금',
 '20세',
 '최악',
 '환경재앙',
 '곳',
 '2부',
 '아랄해',
 '두',
 '물줄기',
 '아무다리아강',
 '시르다리아강',
 '발원지',
 '여정',
 '주요',
 '에피소드',
 '아무다리아강',
 '발원지',
 '파미르고원',
 '대륙빙하',
 '중',
 '최대',
 '규모',
 '파미르의',
 '페드첸코',
 '빙하',
 '판즈강',
 '아무다리아',
 '강',
 '타지키스탄',
 '아프가니스탄',
 '국경',
 '강',
 '판즈강',
 '수',
 '아프가니스탄',
 '사람',
 '모습',
 '수',
 '빙하',
 '습지대',
 '식생',
 '변화',
 '환경',
 '변화',
 '강',
 '사람',
 '삶',
 '소개',
 '시르다리아강',
 '발원지',
 '텐샨산맥',
 '파미르',
 '고원',
 '고도',
 '텐샨산맥',
 '풍경',
 '자랑',
 '초록',
 '산',
 '색색',
 '꽃',
 '화',
 '텐샨',
 '시르다리야강',
 '발원지',
 '텐샨',
 '빙하',
 '모습',
 '소개',
 '자일로여름',
 '초목지',
 '유르트',
 '양',
 '키르',
 '유목민',
 '텐샨산맥',
 '야생동물',
 '한',
 '번',
 '공개',
 '적',
 '비밀',
 '땅',
 '세계',
 '지붕',
 '파미르',
 '거대산맥들',
 '해발',
 '4000m',
 '고지세계',
 '최초',
 '하나',
 '먹이',
 '벌',
 '최상위',
 '포식자',
 '신경전총',
 '제작기간',
 '2년',
 '추적',
 '잠복',
 '제작진',
 '집요함',
 '대작',
 '파미르고원',
 '번',
 '적',
 '익숙',
 '이름',
 '국내',
 '소개',
 '적',
 '비밀',
 '땅',
 '파미르고원',
 '히말라야산맥',
 '비롯',
 '텐샨',
 '카

In [135]:
# 에피소드 제목의 명사를 계산에 사용?
#불용어 제거 안함
epilist = metadata['epi']
han.nouns(epilist[925])

['1960년대',
 '몽골',
 '전통악기',
 '야탁',
 '부활',
 '북한',
 '가야금',
 '연주자',
 '김종암',
 '자취',
 '방송',
 '사상',
 '처음',
 '공개',
 '몽골전통악기',
 '야탁',
 '가야금',
 '한몽',
 '간',
 '우정',
 '확인',
 '공동기획',
 '문화관광부',
 '제작',
 '지원한한몽',
 '공동제작',
 '프로그램',
 '3월',
 '3일월',
 '4일',
 '동안',
 '방영',
 '공동기획',
 '문화관광부',
 '제작',
 '지원',
 '한몽',
 '공동제작',
 '프로그램',
 '3월',
 '3일월',
 '4일',
 '방영',
 '몽골',
 '편',
 '한국',
 '몽골',
 '문화적',
 '이해',
 '도모',
 '한국의',
 '이미지',
 '고양',
 '한류',
 '지속',
 '확산',
 '도모',
 '몽골',
 '쌍방향',
 '공동제작',
 '프로그램',
 '몽골',
 '전통문화',
 '삶',
 '한국과',
 '교류',
 '변화들',
 '몽골',
 '사람',
 '한국',
 '솔롱고스',
 '몽골어로',
 '무지개',
 '나라”',
 '라',
 '형제국',
 '최근',
 '아시아',
 '각처',
 '한류韓流',
 '속',
 '한국',
 '친근',
 '여기',
 '몽골',
 '사람',
 '한국',
 '한국',
 '일',
 '한국',
 '동경',
 '것',
 '몽골반점',
 '우랄알타',
 '어족',
 '등',
 '공통점',
 '칭기즈칸',
 '원',
 '제국',
 '한반',
 '문화',
 '고려양高麗樣',
 '칭송',
 '만큼',
 '한국',
 '몽골',
 '역사적',
 '문화적',
 '연대감',
 '오늘날',
 '몽골',
 '한반도',
 '7배',
 '말큼',
 '국토',
 '세계',
 '여덟',
 '번',
 '자원',
 '풍부',
 '인구',
 '283',
 '명',
 '불과',
 '나라',
 '성장',
 '가능성',
 '나라',
 '지목',
 '한국',
 '몽골',
 '양국',
 '성

In [84]:
len(synoplist)

926

In [85]:
s_nounslist = []

for synop in synoplist:
    s_nouns = ' '.join(han.nouns(synop))
    #print(s_nouns)
    s_nounslist.append(s_nouns)


In [86]:
s_nounslist[0]

'아시아 고산지대 파미르 텐샨히말라야 의 자연 사람 삶 중앙아시아 거대 호수 아랄해 지금 20세 최악 환경재앙 곳 2부 아랄해 두 물줄기 아무다리아강 시르다리아강 발원지 여정 주요 에피소드 아무다리아강 발원지 파미르고원 대륙빙하 중 최대 규모 파미르의 페드첸코 빙하 판즈강 아무다리아 강 타지키스탄 아프가니스탄 국경 강 판즈강 수 아프가니스탄 사람 모습 수 빙하 습지대 식생 변화 환경 변화 강 사람 삶 소개 시르다리아강 발원지 텐샨산맥 파미르 고원 고도 텐샨산맥 풍경 자랑 초록 산 색색 꽃 화 텐샨 시르다리야강 발원지 텐샨 빙하 모습 소개 자일로여름 초목지 유르트 양 키르 유목민 텐샨산맥 야생동물 한 번 공개 적 비밀 땅 세계 지붕 파미르 거대산맥들 해발 4000m 고지세계 최초 하나 먹이 벌 최상위 포식자 신경전총 제작기간 2년 추적 잠복 제작진 집요함 대작 파미르고원 번 적 익숙 이름 국내 소개 적 비밀 땅 파미르고원 히말라야산맥 비롯 텐샨 카라코람 쿤룬 힌두쿠시 산맥 곳 평균 해발고도 4000m 산악 들 히말라야 누구 손길 미지 땅 삭막 땅 발 생명들 소개 자연다큐멘터리 명 답 수 야생동물들 모습 무 카메라 설치 촬영 영상 늑대무리 위계질서 먹이 포식자 면모 수 눈표범 늑대 장면 녀석들 제작진 집요함 지 반증 진귀 영상 사냥 마르코폴로양 눈표범 대가 전리품 늑대 둘 신경전 포착 촬영 외 아이벡스야생 염소 마르코폴로산양 수염 수리 등 파미르 서식 다양한 야생동물 생태 밀 다큐 본방송 2014 06 1618 밤 9시 50분재방송 2014 06 22 밤 8시 15분 3부작 연속 방영제작진제작진 기획 추덕담연출 서준기획의도세계 지붕 파미르 고원 한번쯤 적 익숙 이름 국내 소개 바 파미르 텐샨 카라코람 산맥 쿤룬 산맥 등 험준 고산들 지점 형성 고원지대 그 세계 지붕 히말라야 매력 지역 파미르 고원 위치해 타지키스탄 아프가니스탄 파키스탄 키르기스스탄 중국 등 나라들 국경 불안 정세 접근 허락 곳 산악인 도전 히말라야 파미르 누구 손길 미지 땅 곳 평균 해발 4000

#### 명사 문장 시놉으로 새 데이터프레임 만들기

In [101]:
metadata = pd.DataFrame({'idx':metadata['idx'],'epi':metadata['epi'], 'n_synop':s_nounslist, 'tag':metadata['tag']})


In [102]:
metadata.head(3)

,idx,epi,n_synop,tag
0,0,비밀의 땅 파미르 2부 비밀의 땅 숨겨진 강,아시아 고산지대 파미르 텐샨히말라야 의 자연 사람 삶 중앙아시아 거대 호수 아랄해 ...,파미르 텐샨 히말라야 사람들 아랄해 아무다리아강 시르다리아강 주요 빙하 타지키스탄 ...
1,1,비밀의 땅 파미르 1부 세계의 지붕,세계 지붕 파미르 서식 다양한 야생동물들 생태소개 1부 파미르 서식 다양한 야생동물...,서식 세계 지붕 파미르 야생 야생동물 지역 포식자 늑대 중심 눈표범 마르코폴 동물 ...
2,2,위대한 로마 제작노트 로마는 하루아침에 이루어지지 않았다,위대한 로마 촬영 기간 총 6주 3주간 튀니지 재연 촬영 나머지 3주 이탈리아 로마...,로마 6주 폼페이 사건 최대 제국 역사 시간 황제 우리


In [33]:
#metadata.to_csv('ebs_nsynop.csv', encoding='utf-8')

# CountVectorizer

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

#1개 이하로 나오는 단어 제외
count = CountVectorizer(min_df=2)
#count_matrix = count.fit_transform(metadata)

In [104]:
count_matrix = count.fit_transform(metadata['n_synop'])

In [105]:
count_matrix.shape

(926, 13934)

## 코사인 유사도 점수 계산

In [106]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [107]:
# index는 dataframe의 idx
indices = pd.Series(metadata.index, index=metadata['idx'])

In [130]:
def get_recommendations(idx, cosine_sim2=cosine_sim2):
    #제목과 일치하는 에피소드 인덱스 가지고옴
    idx = indices[idx]
    
    sim_scores = list(enumerate(cosine_sim2[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #비슷한 에피소드 상위 10개
    sim_scores = sim_scores[1:11]
    
    # indices
    epi_indices = [i[0] for i in sim_scores]
    
        
    return metadata['epi'].iloc[epi_indices]

### 나중에 epi & recommedation epi, index 한번에 출력 함수 만들기

In [153]:
print('episode : ' + metadata['epi'][236])
get_recommendations(236, cosine_sim2)

episode : 우리 WE 6부 함께 산다는 것


238         우리 WE 4부 끝나지 않은 전쟁 한국의 지역감정을 말하다
239               우리 WE 3부 우리는 왜 우리가 되지 못하는가
237               우리 WE 5부 나일 수도 있었던  나의 이야기
240             우리 WE 2부 대중 유혹의 기술  보이지 않는 손
241    우리 WE 1부 대중 유혹의 기술  그들은 어떻게 우리를 유혹했을까
315                     가족쇼크  9부 엄마의 땅. 키리위나
215                                커뮤니케이션의 힘
730      한일강제병합 100년 특별기획 잊혀진나라 13년 1부 제국의 꿈
729     한일강제병합 100년 특별기획 잊혀진나라 13년 2부 제국의 전쟁
500                    행복의 조건 복지국가를 가다 3부 주거
Name: epi, dtype: object

In [154]:
print('episode : '+ metadata['epi'][400])
get_recommendations(400, cosine_sim2)

episode : 수학의 위대한 여정 1부 미스터리 소수


399    수학의 위대한 여정 2부 세상을 바꾸는 힘 방정식
116               넘버스  5부 천공의 수 허수
119             넘버스  2부 천국의 사다리 ∞ 
118               넘버스  3부 자유의 수 엑스
120               넘버스  1부 하늘의 수  π
117                넘버스  4부 신의 손짓 영
345             문명과 수학  5부 남겨진 문제들
348                  문명과 수학  2부 원론
347               문명과 수학  3부 신의 숫자
346         문명과 수학  4부 움직이는 세계 미적분
Name: epi, dtype: object

In [148]:
def recommender(idx):
    print('episode : ' + metadata['epi'][idx])
    
    get_recommendations(idx, cosine_sim2)
    

In [149]:
recommender(0)

episode : 비밀의 땅 파미르   2부 비밀의 땅 숨겨진 강
